In [25]:
import networkx as nx
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

# Paso 1: Leer el archivo y construir el grafo
def cargar_grafo():
    G = nx.Graph()
    ciudades = set()  # Usar un conjunto para evitar duplicados en ciudades
    with open('distancias.txt', 'r') as file:
        for line in file:
            origen, destino, distancia = line.strip().split(',')
            distancia = int(distancia)
            G.add_edge(origen, destino, weight=distancia)
            ciudades.add(origen)
            ciudades.add(destino)
    return G, list(ciudades)

# Paso 2: Crear el grafo con networkx
grafo, lista_ciudades = cargar_grafo()

# Paso 3: Implementar Dijkstra para encontrar la distancia mínima
def calcular_ruta_mas_corta(grafo, origen, destino):
    try:
        distancia, ruta = nx.single_source_dijkstra(grafo, origen, destino, weight='weight')
        return distancia, ruta
    except nx.NetworkXNoPath:
        return None, []

# Paso 4: Crear la interfaz gráfica
def mostrar_interfaz():
    def mostrar_resultado():
        # Limpiar la tabla de resultados anterior
        for item in ruta_tree.get_children():
            ruta_tree.delete(item)
        
        origen = combo_origen.get()
        destino = combo_destino.get()
        if origen == destino:
            messagebox.showerror("Error", "La ciudad de origen y destino no pueden ser la misma.")
            return

        distancia_total, ruta = calcular_ruta_mas_corta(grafo, origen, destino)
        if ruta:
            # Agregar la ciudad de origen con distancia acumulada 0
            ruta_tree.insert("", "end", values=(origen, 0))
            
            # Llenar la tabla con el detalle de la ruta acumulando las distancias
            distancia_acumulada = 0
            for i in range(len(ruta) - 1):
                ciudad1 = ruta[i]
                ciudad2 = ruta[i + 1]
                distancia = grafo[ciudad1][ciudad2]["weight"]
                distancia_acumulada += distancia
                ruta_tree.insert("", "end", values=(ciudad2, distancia_acumulada))

            # Mostrar la distancia total
            total_label.config(text=f"Distancia total: {distancia_acumulada} km")
        else:
            total_label.config(text="No existe una ruta entre las ciudades seleccionadas.")

    # Configuración de la ventana principal
    ventana = tk.Tk()
    ventana.title("Calculador de Ruta Más Corta")
    ventana.geometry("800x550")  # Ajustar el tamaño de la ventana para una mejor distribución

    # Frame para organizar las tablas de Ciudades y Aristas en una fila
    frame_tablas = tk.Frame(ventana)
    frame_tablas.pack(pady=5)

    # Tabla para mostrar la lista de ciudades (vértices)
    frame_ciudades = tk.Frame(frame_tablas)
    frame_ciudades.pack(side="left", padx=5)
    tk.Label(frame_ciudades, text="Lista de Ciudades (Vértices):").pack(pady=2)
    ciudades_tree = ttk.Treeview(frame_ciudades, columns=("Nombre",), show="headings", height=8)
    ciudades_tree.heading("Nombre", text="Nombre")
    for ciudad in lista_ciudades:
        ciudades_tree.insert("", "end", values=(ciudad,))
    ciudades_tree.pack()

    # Tabla para mostrar la lista de distancias entre ciudades adyacentes (aristas)
    frame_aristas = tk.Frame(frame_tablas)
    frame_aristas.pack(side="left", padx=5)
    tk.Label(frame_aristas, text="Lista de Distancias entre Ciudades Adyacentes (Aristas):").pack(pady=2)
    aristas_tree = ttk.Treeview(frame_aristas, columns=("Ciudad1", "Ciudad2", "Distancia"), show="headings", height=8)
    aristas_tree.heading("Ciudad1", text="Ciudad 1")
    aristas_tree.heading("Ciudad2", text="Ciudad 2")
    aristas_tree.heading("Distancia", text="Distancia (km)")
    for u, v, d in grafo.edges(data=True):
        aristas_tree.insert("", "end", values=(u, v, d["weight"]))
    aristas_tree.pack()

    # Dropdowns y botón para seleccionar ciudades y calcular la ruta
    frame_seleccion = tk.Frame(ventana)
    frame_seleccion.pack(pady=5)
    tk.Label(frame_seleccion, text="Ciudad Origen:").grid(row=0, column=0, padx=5, pady=2)
    combo_origen = ttk.Combobox(frame_seleccion, values=lista_ciudades, width=15)
    combo_origen.grid(row=0, column=1, padx=5, pady=2)
    tk.Label(frame_seleccion, text="Ciudad Destino:").grid(row=1, column=0, padx=5, pady=2)
    combo_destino = ttk.Combobox(frame_seleccion, values=lista_ciudades, width=15)
    combo_destino.grid(row=1, column=1, padx=5, pady=2)
    boton_calcular = tk.Button(frame_seleccion, text="Calcular Ruta Más Corta", command=mostrar_resultado)
    boton_calcular.grid(row=2, column=0, columnspan=2, pady=5)

    # Tabla para mostrar la ruta más corta y distancias acumuladas
    frame_ruta = tk.Frame(ventana)
    frame_ruta.pack(pady=5)
    tk.Label(frame_ruta, text="Ruta Más Corta con Distancias Acumuladas:").pack(pady=2)
    ruta_tree = ttk.Treeview(frame_ruta, columns=("Ciudad", "Distancia Acumulada"), show="headings", height=6)
    ruta_tree.heading("Ciudad", text="Ciudad")
    ruta_tree.heading("Distancia Acumulada", text="Distancia Acumulada (km)")
    ruta_tree.pack(pady=2)

    # Label para mostrar la distancia total
    total_label = tk.Label(ventana, text="")
    total_label.pack(pady=5)

    ventana.mainloop()

# Ejecutar la interfaz
mostrar_interfaz()
